## Maps of pages
Mappiamo le pagine a disposizione e le loro caratteristiche. Ogni blocco di codice prova a rispondere a una domanda.

In [97]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd   
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt

interactions = pd.read_csv('/home/jacoponudo/Documents/Size_effects/DATA/interacions/facebook_outreach_vs_interaction.csv', usecols=['post_id', 'alpha', 'outreach'])
posts = pd.read_csv('/home/jacoponudo/Documents/Size_effects/DATA/sample_posts.csv', usecols=['created_time', 'page_id', 'post_id'], encoding='ISO-8859-1')

posts_dict = posts.set_index('post_id').T.to_dict()

interactions['created_time'] = interactions['post_id'].map(lambda x: posts_dict.get(x, {}).get('created_time', None))
interactions['page_id'] = interactions['post_id'].map(lambda x: posts_dict.get(x, {}).get('page_id', None))

interactions=interactions.dropna()
print(interactions.head())

/tmp/ipykernel_8169/577968565.py:11: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  posts_dict = posts.set_index('post_id').T.to_dict()


                            post_id  alpha   outreach  \
1     35281584398_10151655784439399   True   1350.125   
3   113349742029506_644708212226987   True      6.500   
4      7155422274_10153792989482275   True  44599.625   
8       67175109350_205981829418171   True     43.125   
11     7155422274_10151218230287275   True  15919.625   

                created_time          page_id  
1   2013-06-10T16:04:00+0000      35281584398  
3   2013-09-30T06:45:27+0000  113349742029506  
4   2015-10-07T19:44:42+0000       7155422274  
8   2011-03-01T17:33:52+0000      67175109350  
11  2012-08-24T08:15:30+0000       7155422274  


Quante pagine ci sono? 

In [3]:
len(interactions['page_id'].unique())

92

Di quante abbiamo la serie storica dei dati di attività dal 2010 al 2015?

In [150]:
outreach = pd.read_csv('/home/jacoponudo/Documents/Size_effects/DATA/sample_outreachs.csv')
pages = outreach.groupby('page_id').size().reset_index(name='TS_coverage(W)')

pagine_con_oltre_200_righe = (pages['TS_coverage(W)'] > 48).sum()
print(f"Pagine con più di 200 righe: {pagine_con_oltre_200_righe}")

Pagine con più di 200 righe: 88


Dividiamo l'outreach iniziale in 3 classi bilanciate.

In [151]:
import pandas as pd

# Filtra le pagine con TS_coverage(W) > 48 e ottieni i page_id
focus = pages[pages['TS_coverage(W)'] > 48]['page_id']

# Filtra le interazioni basandosi sui page_id di focus
interactions = interactions[interactions['page_id'].isin(focus)]

# Ordina le interazioni in base a created_time
interactions = interactions.sort_values(by='created_time')

# Seleziona solo le prime 100 righe in ordine cronologico per ogni page_id
top_interactions = interactions.groupby('page_id').head(100)

# Trova il primo post e il valore di outreach corrispondente per ogni pagina
first_outreach = top_interactions.loc[top_interactions.groupby('page_id')['created_time'].idxmin()]

# Crea le classi omogenee per l'outreach usando i quantili
quantiles = first_outreach['outreach'].quantile([0, 0.25, 0.5, 0.75, 1]).values
labels = ['Molto Basso', 'Basso', 'Medio', 'Alto']
first_outreach['outreach_binned'] = pd.cut(first_outreach['outreach'], bins=quantiles, labels=labels, include_lowest=True)

# Mantieni solo le righe uniche nel DataFrame 'first_outreach' per il merge
first_outreach_unique = first_outreach[['page_id', 'outreach_binned','outreach']].drop_duplicates('page_id')

# Aggiungi la categoria al DataFrame 'pages' usando un merge
pages = pages.merge(first_outreach_unique, on='page_id', how='left')

# Stampa i limiti delle classi
for label, lower, upper in zip(labels, quantiles[:-1], quantiles[1:]):
    print(f"Classe: {label}, Estremo inferiore: {lower:.2f}, Estremo superiore: {upper:.2f}")

# Rinomina le colonne di 'pages' se necessario
pages.columns = ['page_id', 'TS_coverage(W)', 'initial_class_size', 'initial_size']


Classe: Molto Basso, Estremo inferiore: 0.00, Estremo superiore: 0.69
Classe: Basso, Estremo inferiore: 0.69, Estremo superiore: 3.25
Classe: Medio, Estremo inferiore: 3.25, Estremo superiore: 12.38
Classe: Alto, Estremo inferiore: 12.38, Estremo superiore: 12147.38


Dividiamo l'outreach finale in 3 classi bilanciate.

In [152]:
# Trova il primo post e il valore di outreach corrispondente per ogni pagina
interactions['created_time'] = pd.to_datetime(interactions['created_time'])
last_outreach = interactions.loc[interactions.groupby('page_id')['created_time'].idxmax()]

# Crea le classi omogenee per l'outreach usando i quantili
quantiles = last_outreach['outreach'].quantile([0, 1/4, 1/2, 3/4, 1]).values
labels = ['Molto Basso', 'Basso', 'Medio', 'Alto']
last_outreach['outreach_binned'] = pd.cut(last_outreach['outreach'], bins=quantiles, labels=labels, include_lowest=True)

# Mantieni solo le righe uniche nel DataFrame 'last_outreach' per il merge
last_outreach_unique = last_outreach[['page_id', 'outreach_binned', 'outreach']].drop_duplicates('page_id')

# Aggiungi la categoria al DataFrame 'pages' usando un merge
pages = pages.merge(last_outreach_unique, on='page_id', how='left')

# Stampa i limiti delle classi
for label, lower, upper in zip(labels, quantiles[:-1], quantiles[1:]):
    print(f"Classe: {label}, Estremo inferiore: {lower:.2f}, Estremo superiore: {upper:.2f}")

# Aggiorna i nomi delle colonne nel DataFrame 'pages'
pages.columns = ['page_id', 'TS_coverage(W)', 'initial_class_size', 'initial_size', 'last_class_size', 'last_size']


Classe: Molto Basso, Estremo inferiore: 0.00, Estremo superiore: 11.56
Classe: Basso, Estremo inferiore: 11.56, Estremo superiore: 168.25
Classe: Medio, Estremo inferiore: 168.25, Estremo superiore: 3163.12
Classe: Alto, Estremo inferiore: 3163.12, Estremo superiore: 428301.75


Qual'è il delta della crescita tra prima e dopo.

In [153]:
pages['delta_growth']=pages['last_size']-pages['initial_size']

In quale arco temporale lavorano.

In [154]:
interactions['created_time'] = pd.to_datetime(interactions['created_time'])
diff_days = interactions.groupby('page_id')['created_time'].agg(max_date='max', min_date='min')
diff_days['diff_days'] = (diff_days['max_date'] - diff_days['min_date']).dt.days
pages = pages.merge(diff_days[['diff_days']], on='page_id', how='left')


In [155]:
pages

,page_id,TS_coverage(W),initial_class_size,initial_size,last_class_size,last_size,delta_growth,diff_days
0,6533373917,306,Molto Basso,0.000,Alto,6003.125,6003.125,2061.0
1,7130088677,291,Molto Basso,0.500,Medio,185.625,185.125,1996.0
2,7155422274,306,Alto,64.750,Alto,47697.250,47632.500,2129.0
3,7711594722,307,Basso,3.250,Alto,3222.750,3219.500,2137.0
4,8260742627,306,Basso,1.750,Basso,16.375,14.625,2126.0
...,...,...,...,...,...,...,...,...
88,527351247377965,95,Medio,4.125,Basso,82.500,78.375,651.0
89,552321618120006,136,NaN,NaN,NaN,NaN,NaN,NaN
90,554092937948891,143,Medio,11.000,Basso,11.875,0.875,969.0
91,842786335748518,5,NaN,NaN,NaN,NaN,NaN,NaN
